# 预处理

参考：[scikit-learn preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html)

`sklearn.preprocessing` 包提供了几个常用的实用函数和变换器类，将原始特征向量转换为更适合下游估计器的表示形式。

一般来说，学习算法受益于数据集的标准化。如果集合中存在一些离群值，则鲁棒的 scalers 或 transformers 更为合适。不同的 scalers, transformers 和 normalizers 在包含边缘性离群值的数据集上的行为在 [比较不同的 scalers 对有离群值的数据的影响](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py) 中得到了强调。

## 标准化，或去均值和方差缩放

数据集的标准化（Standardization）是许多在 scikit-learn 中实现的 **机器学习估计器（estimators）的共同需求**；如果单个特征或多或少不像标准的正态分布数据：**零均值和单位方差** （zero mean and unit variance）的高斯分布数据，那么它们可能表现得很糟糕。

在实践中，经常忽略分布的形状，只是通过移除每个特征的平均值来将数据变换为中心，然后通过将非常数特征除以它们的标准差来缩放。

例如，在学习算法的目标函数中使用的许多元素（如支持向量机（Support Vector Machines）的 RBF 核或线性模型的 l1 和 l2 正则化器（regularizers））可能假设所有特征都以零为中心或具有相同的方差顺序。如果特征的方差比其他特征大几个数量级，它可能会支配目标函数，使估计器无法像预期的那样正确地从其他特征学习。

{mod}`sklearn.preprocessing` 模块提供了 {class}`~sklearn.preprocessing.StandardScaler` 实用程序类，它是对类数组数据集执行以下操作的一种快速而简单的方法：

In [1]:
from sklearn import preprocessing
import numpy as np

X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
scaler = preprocessing.StandardScaler().fit(X_train)
scaler

StandardScaler()

In [2]:
scaler.mean_

array([1.        , 0.        , 0.33333333])

In [3]:
scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

In [4]:
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

缩放后数据的均值为零和方差为 1：

In [5]:
X_scaled.mean(axis=0), X_scaled.std(axis=0)

(array([0., 0., 0.]), array([1., 1., 1.]))

这个类实现了 `Transformer` API 来计算训练集上的均值和标准差，以便以后能够在测试集上重新应用相同的变换。因此，这个类适合在  `sklearn.pipeline` 的早期步骤中使用：

In [6]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X, y = make_classification(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # 对训练数据进行缩放

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

在测试数据上应用缩放，不泄露训练数据。

In [7]:
pipe.score(X_test, y_test)

0.96

通过向 {class}`~sklearn.preprocessing.StandardScaler` 的构造函数传递 `with_mean=False` 或 `with_std=False`，可以禁用居中或缩放。

## 量化

离散化（Discretization，也称为量化（quantization）或分箱（binning））提供了一种将连续特征划分为离散值的方法。某些具有连续特征的数据集可能从离散化中受益，因为离散化可以将具有连续属性的数据集转换为只有名义（nominal）属性的数据集。

One-hot 编码的离散化特征可以使模型更具表达性，同时保持可解释性。例如，使用离散器的预处理可以为线性模型引入非线性。

### K-bins 离散化

{class}`~sklearn.preprocessing.KBinsDiscretizer` 将特征离散到 k 个箱子中：

In [9]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 2], encode='ordinal').fit(X)

默认情况下，输出是 one-hot 编码到稀疏矩阵（参见 [编码分类特征](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)），这可以通过 `encode` 参数配置。对于每个特征，箱边在 `fit` 期间计算，并与箱的数量一起，它们将定义区间。因此，对于当前示例，这些区间定义为：